In [85]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression,LogisticRegressionCV
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor, RandomForestClassifier,GradientBoostingClassifier
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error, accuracy_score, f1_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split, cross_val_score

from sklearn.svm import SVC

from sklearn import svm
from sklearn.metrics import accuracy_score, precision_score, recall_score
import pickle

# rf = RandomForestRegressor()

In [2]:
mrna_df =pd.read_csv('/Users/cp/Desktop/capstone2.mrn_df2.csv')

In [3]:
df = pd.read_csv('~/Downloads/METABRIC_RNA_Mutation.csv')

/Users/cp/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3147: DtypeWarning: Columns (678,688,690,692) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
death_from_dict = {
    'Living':0
    ,'Died of Other Causes':0
    ,'Died of Disease':1
}
df.replace(death_from_dict, inplace =True)

In [5]:
df.death_from_cancer

0       0.0
1       0.0
2       1.0
3       0.0
4       1.0
       ... 
1899    0.0
1900    1.0
1901    1.0
1902    0.0
1903    0.0
Name: death_from_cancer, Length: 1904, dtype: float64

In [6]:
mrna_df['death_from_cancer'] = df.death_from_cancer

In [7]:
mrna_df.death_from_cancer.isnull().sum()


1

In [8]:
mrna_df.death_from_cancer.fillna(0, inplace = True)

In [9]:
mrna_df.death_from_cancer.isnull().sum()


0

In [10]:
mrna_df.shape

(1904, 490)

In [11]:
y = mrna_df.pop('death_from_cancer')

In [12]:
X = mrna_df

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.10, random_state=1)


# X_full, X_val, y_full, y_val = train_test_split(X, y, test_size=0.10, random_state=42, stratify = y)

In [20]:
# y.isnull().sum()
X.isnull().sum().sum()

0

In [21]:
X.info(verbose= True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1904 entries, 0 to 1903
Data columns (total 489 columns):
 #   Column     Dtype  
---  ------     -----  
 0   brca1      float64
 1   brca2      float64
 2   palb2      float64
 3   pten       float64
 4   tp53       float64
 5   atm        float64
 6   cdh1       float64
 7   chek2      float64
 8   nbn        float64
 9   nf1        float64
 10  stk11      float64
 11  bard1      float64
 12  mlh1       float64
 13  msh2       float64
 14  msh6       float64
 15  pms2       float64
 16  epcam      float64
 17  rad51c     float64
 18  rad51d     float64
 19  rad50      float64
 20  rb1        float64
 21  rbl1       float64
 22  rbl2       float64
 23  ccna1      float64
 24  ccnb1      float64
 25  cdk1       float64
 26  ccne1      float64
 27  cdk2       float64
 28  cdc25a     float64
 29  ccnd1      float64
 30  cdk4       float64
 31  cdk6       float64
 32  ccnd2      float64
 33  cdkn2a     float64
 34  cdkn2b     float64
 35 

In [ ]:
mrna_df.info(verbose= True)

In [26]:
lst = X_train.shape, X_test.shape, X_val.shape
print(lst)

((1456, 489), (286, 489), (162, 489))


In [37]:
kfold = KFold(n_splits=15)

accuracies = []
precisions = []
recalls = []

# for num in range(1, 30):
#     kfold = KFold(n_splits=num)
for train_index, test_index in kfold.split(X_train):
    model = LogisticRegression(solver="lbfgs", max_iter=500)
    model.fit(X_train.iloc[train_index], y_train.iloc[train_index])
    y_predict = model.predict(X_train.iloc[test_index])
    y_true = y_train.iloc[test_index]
    accuracies.append(accuracy_score(y_true, y_predict))
    precisions.append(precision_score(y_true, y_predict))
    recalls.append(recall_score(y_true, y_predict))

print("Accuracy:", np.average(accuracies))
print("Precision:", np.average(precisions))
print("Recall:", np.average(recalls))

Accuracy: 0.598204642681815
Precision: 0.40437683835732713
Recall: 0.4226006219160451


In [ ]:
# X_train.shape
# X_test.shape
# y_train.shape

# y_train.shape

In [35]:
lr = LogisticRegression(random_state = 1)

In [45]:
model = svm.SVC(kernel='rbf',C=30,gamma='auto')
model.fit(X_train,y_train)
model.score(X_test, y_test)


0.6678321678321678

In [46]:
model_params = {
    'svm': {
        'model': svm.SVC(gamma='auto'),
        'params' : {
            'C': [1,10,20],
            'kernel': ['rbf','linear']
        }  
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [1,5,10]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'C': [1,5,10]
        }
    }
}   

In [48]:

scores = []

for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(X_test, y_test)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    
df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

,model,best_score,best_params
0,svm,0.702783,"{'C': 1, 'kernel': 'rbf'}"
1,random_forest,0.664247,{'n_estimators': 10}
2,logistic_regression,0.601331,{'C': 1}


In [76]:
model_params = {
    'svm': {
        'model': svm.SVC(gamma='auto'),
        'params' : {
            'C': [1,10,20],
            'kernel': ['rbf','linear']
        }  
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [1,5,10]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'C': [1,5,10],
            'penalty':['l1', 'l2']
#             'l1_ratio':[]
        }
    },
    'Gradient Tree Boosting': {
        'model':GradientBoostingClassifier(),
        'params':{
            'n_estimators':[100, 250, 500],
            'learning_rate':[0.1, 0.2],
            'max_depth' : [2,4, 8]
            
            
        }
    }
#     'LogisticRegressionCV':{
#         'model':LogisticRegressionCV(),
#         'params':{
#             'Cs':[5, 10, 20]
#             ,'cv':[5,10]
#             ,'max_iter'= [100, 200]
#         }
#     }
}   

In [84]:
scores = []

for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(X_test, y_test)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    
df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

,model,best_score,best_params
0,svm,0.702783,"{'C': 1, 'kernel': 'rbf'}"
1,random_forest,0.671385,{'n_estimators': 10}
2,logistic_regression,0.625650,"{'C': 1, 'penalty': 'l1'}"
3,Gradient Tree Boosting,0.660920,"{'learning_rate': 0.2, 'max_depth': 4, 'n_esti..."
4,LogisticRegressionCV,0.702783,"{'Cs': 5, 'cv': 5, 'max_iter': 100}"


In [83]:
model_params = {
    'svm': {
        'model': svm.SVC(gamma='auto'),
        'params' : {
            'C': [1,10,20],
            'kernel': ['rbf','linear']
        }  
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [1,5,10]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'C': [1,5,10],
            'penalty':['l1', 'l2']
#             'l1_ratio':[]
        }
    },
    'Gradient Tree Boosting': {
        'model':GradientBoostingClassifier(),
        'params':{
            'n_estimators':[100, 250, 500],
            'learning_rate':[0.1, 0.2],
            'max_depth' : [2,4, 8]
            
            
        }
    },
        'LogisticRegressionCV':{
        'model':LogisticRegressionCV(),
        'params':{
            'Cs':[5, 10, 20]
            ,'cv':[5,10]
            ,'max_iter': [100, 200]
        }
    }
}

In [88]:
lrcv = LogisticRegressionCV(Cs = 5, cv =5, max_iter = 100).fit(X_train, y_train)


/Users/cp/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/cp/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model